<a href="https://colab.research.google.com/github/Ishaan1977/FakeNewsDetection-/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
import string

In [27]:
dset_fake = pd.read_csv('/Fake.csv')
dset_real = pd.read_csv('/True.csv')

In [28]:
dset_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [29]:
dset_real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [30]:
dset_fake['class'] = 0
dset_real['class'] = 1

In [31]:
dset_fake.shape, dset_real.shape

((23481, 5), (21417, 5))

In [32]:
df_fake_manual_test = dset_fake.tail(10)
for i in range(23480, 23470, -1):
  dset_fake.drop([i], axis=0, inplace=True, errors='ignore')

df_real_manual_test = dset_real.tail(10)
for i in range(21417, 21407, -1):
  dset_real.drop([i], axis=0, inplace=True, errors='ignore')


In [33]:
dset_fake.shape, dset_real.shape

((23471, 5), (21408, 5))

In [34]:
df_fake_manual_test['class'] = 0
df_real_manual_test['class'] = 1

<ipython-input-34-f53571640d14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_test['class'] = 0
<ipython-input-34-f53571640d14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real_manual_test['class'] = 1


In [36]:
df_fake_manual_test.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [37]:
df_real_manual_test.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [38]:
df_merge = pd.concat([dset_fake, dset_real], axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [39]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [40]:
data = df_merge.drop(['title', 'subject', 'date'], axis=1) #dropping the nonn reqd columns

In [41]:
data.isnull().sum()

,0
text,0
class,0


no null values

In [42]:
data = data.sample(frac=1) # random shuffling of data

In [43]:
data.head()

,text,class
1973,WASHINGTON (Reuters) - The Interior Department...,1
5014,WASHINGTON (Reuters) - President Donald Trump ...,1
14626,You re gonna love this patriot! He speaks for ...,0
20564,It s interesting how much vitriol some women h...,0
5908,As if there wasn t enough reasons to hate Dona...,0


In [45]:
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

In [46]:
data.columns

Index(['level_0', 'text', 'class'], dtype='object')

In [47]:
data.head()

,level_0,text,class
0,0,WASHINGTON (Reuters) - The Interior Department...,1
1,1,WASHINGTON (Reuters) - President Donald Trump ...,1
2,2,You re gonna love this patriot! He speaks for ...,0
3,3,It s interesting how much vitriol some women h...,0
4,4,As if there wasn t enough reasons to hate Dona...,0


In [48]:
data.drop(['level_0'], axis=1, inplace=True)

In [49]:
data.head()

,text,class
0,WASHINGTON (Reuters) - The Interior Department...,1
1,WASHINGTON (Reuters) - President Donald Trump ...,1
2,You re gonna love this patriot! He speaks for ...,0
3,It s interesting how much vitriol some women h...,0
4,As if there wasn t enough reasons to hate Dona...,0


In [50]:
def optm(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text


In [51]:
data['text'] = data['text'].apply(optm)

In [53]:
X = data['text']
y = data['class']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer  # used to convert text to vectors
vectorize = TfidfVectorizer()
Xv_train = vectorize.fit_transform(X_train)
Xv_test = vectorize.transform(X_test)

In [56]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(Xv_train, y_train)

LogisticRegression()

In [57]:
pred_lr = LR.predict(Xv_test)


In [58]:
LR.score(Xv_test, y_test)

0.9861853832442068

In [59]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4661
           1       0.98      0.99      0.99      4315

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



In [60]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(Xv_train, y_train)

DecisionTreeClassifier()

In [62]:
pred_dt = dt.predict(Xv_test)

In [63]:
dt.score(Xv_test, y_test)

0.9961007130124777

In [64]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      4661
           1       1.00      0.99      1.00      4315

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



In [65]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=0)
gb.fit(Xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [66]:
pred_gb = gb.predict(Xv_test)

In [67]:
gb.score(Xv_test, y_test)

0.9963235294117647

In [70]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4661
           1       0.99      1.00      1.00      4315

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



In [71]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(Xv_train, y_train)

RandomForestClassifier()

In [72]:
pred_rf = rf.predict(Xv_test)

In [73]:
rf.score(Xv_test, y_test)

0.9918672014260249

In [74]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4661
           1       0.99      0.99      0.99      4315

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976

